## Initialize the libraries and origin and destination workspaces

In [1]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry, Region

# DESTINATION Keys
dest_training_key = "970bf90c200d4f429a233dee4239e8f1"
dest_prediction_key = "23287e52d75349138403089743a504ab"


# ORIGIN Keys
training_key = "970bf90c200d4f429a233dee4239e8f1"
prediction_key = "23287e52d75349138403089743a504ab"

trainer = training_api.TrainingApi(training_key)
dest_trainer = training_api.TrainingApi(dest_training_key)

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection")
dest_obj_detection_domain = next(domain for domain in dest_trainer.get_domains() if domain.type == "ObjectDetection")

## Get the origin project ID reference

In [2]:
myProjects = trainer.get_projects()

In [3]:
for project in myProjects:
    print(project.name)
    print(project.id)
    print(project.description)

PowerItems_Detection
70e7718a-e1f1-45d5-bba1-60b224f55365
Model to detect Power line objects


In [4]:
Project = trainer.get_project(project_id="70e7718a-e1f1-45d5-bba1-60b224f55365")

## Create the destination Project 

In [5]:
dest_Project = dest_trainer.create_project("PowerItems_Detection_Copy", domain_id=dest_obj_detection_domain.id)

In [6]:
for project in dest_trainer.get_projects():
    print(project.name)
    print(project.id)
    print(project.description)

PowerItems_Detection_Copy
850f7aa1-725c-49f7-b897-af8fc8d038fe

PowerItems_Detection
70e7718a-e1f1-45d5-bba1-60b224f55365
Model to detect Power line objects


## Get the tags on origin project and create same tags on destination project

In [7]:
dest_tags = []
for tag in trainer.get_tags(Project.id):
    dest_tags.append(dest_trainer.create_tag(dest_Project.id, tag.name))
    print(tag.name)

crossarm
pole
insulator
switch
transformer


In [8]:
dest_tags_dict = {}
dest_tag_ids = []
for tag in dest_tags:
    dest_tags_dict[tag.name] = tag.id
    dest_tag_ids.append(tag.id)

print(dest_tags_dict)
print(dest_tag_ids)

{'crossarm': '341c780e-b29c-4c07-973f-37a42593e3bb', 'transformer': '5aa533cb-7237-48a8-a271-7376e733b775', 'insulator': '8e5fab81-9335-461e-a2d5-35ad8f4bd1f9', 'pole': '8b4660eb-5803-4182-957c-e7c600ef6e19', 'switch': 'fce8fe21-89d2-4c9d-8e4d-1ce497d8d5fb'}
['341c780e-b29c-4c07-973f-37a42593e3bb', '8b4660eb-5803-4182-957c-e7c600ef6e19', '8e5fab81-9335-461e-a2d5-35ad8f4bd1f9', 'fce8fe21-89d2-4c9d-8e4d-1ce497d8d5fb', '5aa533cb-7237-48a8-a271-7376e733b775']


## Get the images with regions on origin project

In [9]:
tagged_images = trainer.get_tagged_images(Project.id, take=trainer.get_tagged_image_count(Project.id))

In [10]:
print ("Adding images...")
tagged_images_with_regions = []
for image in tagged_images:
    dest_regions = []
    
    for region in image.regions:
        dest_regions.append(Region(tag_id=dest_tags_dict[region.tag_name], left=region.left, top=region.top, width=region.width, height=region.height))

    tagged_images_with_regions.append(ImageUrlCreateEntry(url=image.image_uri, tag_ids=dest_tag_ids, regions=dest_regions))
print("Done")

Adding images...
Done


## Create the images with regions on destination project

In [11]:
limit = 64 # this is a limit imposed on the API, so we need to batch the creation process
count_of_images = len(tagged_images_with_regions)

for i in range(0,count_of_images,limit):
    begin=i
    end=limit+i
    if(end > count_of_images ): end = count_of_images
    dest_trainer.create_images_from_urls(dest_Project.id, images=tagged_images_with_regions[begin:end])

In [12]:
print("Count of Tagged images on destination project: " + str(trainer.get_tagged_image_count(Project.id)))
print("Count of Tagged images on destination project: " + str(dest_trainer.get_tagged_image_count(dest_Project.id)))

Count of Tagged images on destination project: 161
Count of Tagged images on destination project: 161


## Train the model on destination Project

In [13]:
import time

print ("Training... (showing status each 10 seconds)")
iteration = dest_trainer.train_project(dest_Project.id)
while (iteration.status != "Completed"):
    iteration = dest_trainer.get_iteration(dest_Project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(10)

# The iteration is now trained. Make it the default project endpoint
dest_trainer.update_iteration(dest_Project.id, iteration.id, is_default=True)
print ("Done!")

Training... (showing status each 10 seconds)
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Completed
Done!


## Get and use the default prediction endpoint

In [14]:
#You can make a web search online for "powerline pole" and get an URL:
url = "https://qtxasset.com/styles/breakpoint_xl_880px_w/s3fs/FierceTelecom-1512405505/power-line.jpg/power-line.jpg"

<div style="width:100%; margin-left:auto; margin-right:auto; margin-bottom:5px; margin-top:17px;">
<img src="https://qtxasset.com/styles/breakpoint_xl_880px_w/s3fs/FierceTelecom-1512405505/power-line.jpg/power-line.jpg" alt="IMAGE" /><br>
</div>

In [ ]:
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

# Now there is a trained endpoint that can be used to make a prediction
dest_predictor = prediction_endpoint.PredictionEndpoint(dest_prediction_key)

# Open the sample image and get back the prediction results.
results = dest_predictor.predict_image_url(dest_Project.id, iteration_id=iteration.id, url=url)

In [16]:
# Display the results.
for prediction in results.predictions:
    if(prediction.probability > 0.2): # only show the prediction with more than 30% threshold probability
        print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100), prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height)

	crossarm: 84.18% 0.06955993 0.127402514 0.6086974 0.188501686
	insulator: 36.78% 0.08503753 0.105132759 0.0558488667 0.09866512
	insulator: 29.66% 0.5926842 0.139620245 0.0359255075 0.100303918
	insulator: 55.25% 0.8356637 0.175214514 0.0554241538 0.112975791
	insulator: 52.13% 0.2076345 0.475562364 0.0474298447 0.104888767
	insulator: 20.84% 0.5068957 0.502736032 0.0600918531 0.123302519
	switch: 22.96% 0.08503753 0.105132759 0.0558488667 0.09866512
	switch: 21.96% 0.228568792 0.6182601 0.06475127 0.110271454
